### Culture Knowledge Graphs Using MELODY and the public SPARQL Endpoint for NFDI4Culture

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json

In [13]:
with open("../files/queries.json") as f:
    queries = json.load(f)

In [14]:
queries = queries["feasible_queries"]

In [15]:
queries

{'count_composition_year': 'PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX schema: <http://schema.org/>\n\nSELECT ?year (COUNT(?composition) AS ?numberOfCompositions)\nWHERE {\n  ?composition rdf:type schema:MusicComposition .\n  ?composition schema:dateCreated ?dateCreated .\n  BIND(YEAR(xsd:dateTime(?dateCreated)) AS ?year)\n}\nGROUP BY ?year\nORDER BY ?year',
 'count_composition_decade': 'PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX schema: <http://schema.org/>\n\nSELECT ?decade (COUNT(?composition) AS ?numberOfCompositions)\nWHERE {\n  ?composition rdf:type schema:MusicComposition .\n  ?composition schema:dateCreated ?dateCreated .\n  BIND(FLOOR(YEAR(xsd:dateTime(?dateCreated))/10)*10 AS ?decade)\n}\nGROUP BY ?decade\nORDER BY ?decade',
 'distinct_years': 'PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-

In [16]:
sparql = SPARQLWrapper("https://nfdi4culture.de/sparql")
sparql.setReturnFormat(JSON)

In [17]:
QUERIES = {
    "n_triplets": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT (COUNT(*) AS ?tripleCount)
        WHERE {
            ?s ?p ?o .
        }
    """,

    "all_types": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?type
        WHERE {
            ?s rdf:type ?type .
        }
        ORDER BY ?type
    """,

    "n_instances_per_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?type (COUNT(?s) AS ?instanceCount)
        WHERE {
            ?s rdf:type ?type .
        }
        GROUP BY ?type
        ORDER BY DESC(?instanceCount)
    """,

    "type_properties": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?type ?property (COUNT(?property) AS ?propertyCount)
        WHERE {
            ?s rdf:type ?type ;
               ?property ?o .
        }
        GROUP BY ?type ?property
        ORDER BY ?type DESC(?propertyCount)
    """,

    "all_properties": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?property
        WHERE {
            ?s ?property ?o .
        }
        ORDER BY ?property
    """,

    "n_instances_per_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?property (COUNT(?s) AS ?instanceCount)
        WHERE {
            ?s ?property ?o .
        }
        GROUP BY ?property
        ORDER BY DESC(?instanceCount)
    """,

    "property_values": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?property ?value (COUNT(?s) AS ?usageCount)
        WHERE {
            ?s ?property ?value .
        }
        GROUP BY ?property ?value
        ORDER BY ?property DESC(?usageCount)
    """,

    "subjects_with_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject ?property ?o .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> .
        }
        ORDER BY ?subject
    """,

    "objects_with_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?p ?object .
            ?object rdf:type <TYPE_URI> .
        }
        ORDER BY ?object
    """,

    "subjects_with_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject ?property <VALUE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            FILTER(?property = <PROPERTY_URI> && ?object = <VALUE_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type_and_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> ;
                     ?property ?o .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_type_and_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            ?object rdf:type <TYPE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type_and_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> ;
                     ?property <VALUE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_type_and_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            ?object rdf:type <TYPE_URI> .
            FILTER(?property = <PROPERTY_URI> && ?object = <VALUE_URI>)
        }
        ORDER BY ?object
    """
}

In [18]:
sparql.setQuery(queries["count_compositions_composer_name_decade"])

In [19]:
try:
    ret = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [23]:
df = pd.DataFrame(ret["results"]["bindings"])
# for each entry, conver entry dict into dict['value']
df = df.applymap(lambda x: x['value'])

/tmp/ipykernel_8323/1270602785.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x['value'])


In [24]:
df

,decade,composer,composerName,numberOfCompositions
0,2000,https://rism.online/people/30004985,Anonymus,108916
1,2000,http://d-nb.info/gnd/118818686,"Palestrina, Giovanni Pierluigi da (1525-1594)",4440
2,2000,http://d-nb.info/gnd/118544489,"Händel, Georg Friedrich (1685-1759)",3552
3,2000,http://d-nb.info/gnd/118547356,"Haydn, Joseph (1732-1809)",3485
4,2000,http://d-nb.info/gnd/118584596,"Mozart, Wolfgang Amadeus (1756-1791)",2758
...,...,...,...,...
9995,2000,https://rism.online/people/30004006,Hartl,1
9996,2000,https://rism.online/people/30017546,"Wenckel, A. H.",1
9997,2000,https://rism.online/people/30022043,"Huber, Anton",1
9998,2000,https://rism.online/people/30008646,"Rocca, Carlo Giorgio (19.sc)",1


In [21]:
bindings = ret["results"]["bindings"]
rows = [{'sub': f"<{b['sub']['value']}>",
         'pred': 'rdf:type' if 'rdf-syntax-ns#type' in b['pred']['value'] else b['pred']['value'],
         'obj': f"<{b['obj']['value']}>"}
        for b in bindings]
df = pd.DataFrame(rows)

KeyError: 'sub'

In [7]:
df

,sub,pred,obj
0,<http://www.openlinksw.com/schemas/virtrdf#Def...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
1,<http://www.openlinksw.com/schemas/virtrdf#Def...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
2,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
3,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
4,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
5,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
6,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
7,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
8,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
9,<http://www.openlinksw.com/virtrdf-data-format...,rdf:type,<http://www.openlinksw.com/schemas/virtrdf#Qua...
